In [1]:
import numpy as np
import os
from Bio import SeqIO #pip install biopython
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [17]:
#step 2
def motif_finder(index):
    def kernel(x):
        #return 1 
        return x 
        #return x**2 
        #return x**2/2
        #return x**(x/2)
        #return x**x 
        
    location = 'data_set_' + str(index) + '/' 
    seqs = []
    for record in SeqIO.parse(os.path.join(location, 'sequences' + ".fasta"), "fasta"):
        seqs.append(str(record.seq))

    with open(os.path.join(location,'motiflength.txt'), 'r') as f:
        ML = int(f.read())
        #f.close()    
    
    
    #find frq-ML pattern
    candidates = {}
    for seq in seqs:
        visited = {}
        for i in range(0,len(seq) - ML):
            if seq[i:i + ML] not in candidates:
                candidates[seq[i:i + ML]] = 1
                visited[seq[i:i + ML]] = 1
            else:
                if seq[i:i + ML] not in visited:
                    visited[seq[i:i + ML]] = 1
                    candidates[seq[i:i + ML]] += 1
                
    sorted_candidates = sorted(candidates.items(), key=lambda x: -x[1])            
    
    #print(sorted_candidates)
    
    min_sup = 4#max(candidates.values()) - 3
    
    #print(min_sup)
    
    frq_ml = [[list(i[0]), i[1]] for i in sorted_candidates if i[1] >= min_sup] #item, weight
    
    #print(frq_ml)
    
    motif = []
    for i in range(ML):
        cols = {}
        sum_ = 0
        for item in frq_ml:
            sum_ += kernel(item[1])
            if item[0][i] not in cols:
                cols[item[0][i]] = kernel(item[1])
            else:
                cols[item[0][i]] += kernel(item[1])
                
        motif.append([
            cols['A']/sum_ if 'A' in cols else 0, 
            cols['C']/sum_ if 'C' in cols else 0, 
            cols['G']/sum_ if 'G' in cols else 0, 
            cols['T']/sum_ if 'T' in cols else 0])
    
    print(motif)
    return motif

In [18]:
def motif_test(index, motif):
    def KL(a, b):
        a = np.asarray(a, dtype=np.float)
        b = np.asarray(b, dtype=np.float)
        return np.sum(np.where(a != 0, a * np.log(a / b), 0))
    
    location = 'data_set_' + str(index) + '/' 
    with open(os.path.join(location,'motif.txt'), 'r') as f:
        x = f.read().strip().split('\n')
        y = [i.strip().split(',')[:-1] for i in x][1:]
        z = [[float(j) for j in i] for i in y]
        #f.close()   
        
    diff = 0
    for i in range(len(motif)):
        for j in range(4):
            diff += abs(motif[i][j] - z[i][j])
    
    diff = 0
    for i in range(len(motif)):
        #print(motif[i], z[i])
        kl = KL(motif[i], z[i]) 
        diff += kl if not np.isinf(kl) else 0 #ingore inf
    #print(motif)
    #print(z)
    return diff
    

In [19]:
scores = []
for i in range(10):
    scores.append(motif_test(i, motif_finder(i)))
print(scores)
print(sum(scores))

[[0, 0.6923076923076923, 0, 0.3076923076923077], [0, 0, 0.38461538461538464, 0.6153846153846154], [0, 0.38461538461538464, 0.3076923076923077, 0.3076923076923077], [0, 0.3076923076923077, 0.6923076923076923, 0], [0, 0.3076923076923077, 0.6923076923076923, 0], [0.3076923076923077, 0.3076923076923077, 0, 0.38461538461538464], [0.6153846153846154, 0, 0.38461538461538464, 0]]
[[0, 0.6666666666666666, 0.3333333333333333, 0], [0, 0, 0.3333333333333333, 0.6666666666666666], [0, 0.3333333333333333, 0, 0.6666666666666666], [0.3333333333333333, 0, 0, 0.6666666666666666], [0, 0.6666666666666666, 0, 0.3333333333333333], [0, 0.3333333333333333, 0, 0.6666666666666666], [0.3333333333333333, 0.3333333333333333, 0, 0.3333333333333333]]
[[0, 0, 1.0, 0], [0, 1.0, 0, 0], [0, 0, 1.0, 0], [0, 0, 1.0, 0], [0, 0, 0, 1.0], [1.0, 0, 0, 0], [0, 0, 1.0, 0]]
[[0.3333333333333333, 0.6666666666666666, 0, 0], [0.3333333333333333, 0.3333333333333333, 0, 0.3333333333333333], [0.6666666666666666, 0.3333333333333333, 0, 

C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """
